In [1]:
%matplotlib inline

import math
import time
import glob
import sys
import cv2, os
import random
import itertools
import numpy as np
import matplotlib.pyplot as plt
import matplotlib.image as im
from PIL import Image
from os import listdir
from os.path import join, basename, isdir
from scikitplot.metrics import plot_confusion_matrix, plot_roc
from sklearn.metrics import confusion_matrix, accuracy_score
from sklearn.metrics import roc_auc_score
from sklearn.utils import shuffle
from skimage.color import gray2rgb
from tqdm import tqdm
from scipy import ndimage
import h5py
import zipfile
import tensorflow as tf
from tensorflow import keras
from tensorflow.keras import layers

In [2]:

import tensorflow as tf
from tensorflow import keras
from tensorflow.keras import optimizers
from tensorflow.keras.optimizers import schedules
import keras
#from keras.datasets import cifar10

from keras.preprocessing.image import ImageDataGenerator
from keras.regularizers import l1, l2

from keras.models import Sequential   #to creat hirachical layers
from keras.layers import Dense, Activation, Dropout, Flatten
from keras.callbacks import ModelCheckpoint
from keras.models import Model, load_model, save_model, Sequential
from keras.layers import Conv3D, MaxPool3D,GlobalAveragePooling3D, Flatten, Dense
from keras.layers import Dropout, Input, BatchNormalization, GRU, Bidirectional,Reshape
from sklearn.metrics import confusion_matrix, accuracy_score
from keras.losses import categorical_crossentropy, binary_crossentropy
from keras.optimizers import Adadelta
from matplotlib.pyplot import cm
from keras.models import Model, load_model, save_model, Sequential
from keras. optimizers import Adam, SGD
from keras.callbacks import TensorBoard, Callback
from keras.utils import np_utils
# more info on callbakcs: https://keras.io/callbacks/ model saver is cool too.
from tensorflow.keras.callbacks import TensorBoard
from keras.backend.tensorflow_backend import set_session

Using TensorFlow backend.


In [4]:

# Reading nii MRI 

import cv2, os
import matplotlib.image as img
import nibabel as nb
from nibabel.testing import data_path
import SimpleITK as sitk

from scipy import ndimage

In [6]:
gpus = tf.config.experimental.list_physical_devices('GPU')
if gpus:
  try:
    # Currently, memory growth needs to be the same across GPUs
    for gpu in gpus:
      tf.config.experimental.set_memory_growth(gpu, True)
    logical_gpus = tf.config.experimental.list_logical_devices('GPU')
    print(len(gpus), "Physical GPUs,", len(logical_gpus), "Logical GPUs")
  except RuntimeError as e:
    # Memory growth must be set before GPUs have been initialized
    print(e)

1 Physical GPUs, 1 Logical GPUs


In [9]:
# code for reading nifit files and resizing data

DATA_DIR = "F:/Python/Dataset/NIMH/Fusion/dev"  # real dataset on PC
train_dir = "F:/Python/Dataset/NIMH/Fusion/dev/train"  # whole dataset on PC


num_classes=6

In [13]:
os.listdir(train_dir)

['Month0', 'Month12', 'Month24', 'Month3', 'Month36', 'Month9']

In [14]:
#os.listdir(Test_dir)

In [15]:

# get all nii files from a directory

interpolation_order = 2
out_shape = (100,100,50)    
training_data = []

CATEGORIES = ["Month0", "Month3","Month9", "Month12","Month24","Month36"]
for category in CATEGORIES:  # do Month0 and Month12 , Month24
    class_label = CATEGORIES.index(category)
    path = os.path.join(train_dir,category)
    print (path)
    
    nii_files = [x for x in os.listdir(path) if 'nii' in x]
   
    # read every file
    for nii_file in nii_files:
        print(os.path.join(path,nii_file))
        # read the nii data

        nii_data = nb.load(os.path.join(path,nii_file))

        # use methods to get information see manual for details (https://nipy.org/nibabel/)
        
        np_data = nii_data.get_data()
        print('Old shape: ', np_data.shape)
        
        # resize data using ndimage from scipy (https://docs.scipy.org/doc/scipy/reference/ndimage.html)
        original_shape =np_data.shape
        assert(len(original_shape) == len(out_shape))

        scale = [(out_shape[i] + 0.0)/original_shape[i] for i in range(len(original_shape))]
        
        print(scale)
        
        out_data = ndimage.interpolation.zoom(np_data, scale, order = interpolation_order)

        print('New shape: ', out_data.shape)
        
        training_data.append([out_data, class_label])  # add this to our training_data
        
        print("length of training data: ",len(training_data))

            

F:/Python/Dataset/NIMH/Fusion/dev/train\Month0
F:/Python/Dataset/NIMH/Fusion/dev/train\Month0\deface_1436_v1_pdw.nii.gz
Old shape:  (192, 256, 40)
New shape:  (100, 100, 50)
length of training data:  1
F:/Python/Dataset/NIMH/Fusion/dev/train\Month0\deface_1436_v1_t1w.nii.gz
Old shape:  (192, 256, 40)
New shape:  (100, 100, 50)
length of training data:  2
F:/Python/Dataset/NIMH/Fusion/dev/train\Month0\deface_1436_v1_t2w.nii.gz
Old shape:  (192, 256, 40)
New shape:  (100, 100, 50)
length of training data:  3
F:/Python/Dataset/NIMH/Fusion/dev/train\Month0\deface_1451_v1_pdw.nii.gz
Old shape:  (192, 256, 48)
New shape:  (100, 100, 50)
length of training data:  4
F:/Python/Dataset/NIMH/Fusion/dev/train\Month0\deface_1451_v1_t1w.nii.gz
Old shape:  (192, 256, 48)
New shape:  (100, 100, 50)
length of training data:  5
F:/Python/Dataset/NIMH/Fusion/dev/train\Month0\deface_1451_v1_t2w.nii.gz
Old shape:  (192, 256, 48)
New shape:  (100, 100, 50)
length of training data:  6
F:/Python/Dataset/NIMH/

New shape:  (100, 100, 50)
length of training data:  53
F:/Python/Dataset/NIMH/Fusion/dev/train\Month0\deface_1532_v1_t2w.nii.gz
Old shape:  (256, 256, 43)
New shape:  (100, 100, 50)
length of training data:  54
F:/Python/Dataset/NIMH/Fusion/dev/train\Month0\deface_1538_v1_pdw.nii.gz
Old shape:  (256, 256, 37)
New shape:  (100, 100, 50)
length of training data:  55
F:/Python/Dataset/NIMH/Fusion/dev/train\Month0\deface_1538_v1_t1w.nii.gz
Old shape:  (256, 256, 37)
New shape:  (100, 100, 50)
length of training data:  56
F:/Python/Dataset/NIMH/Fusion/dev/train\Month0\deface_1538_v1_t2w.nii.gz
Old shape:  (256, 256, 37)
New shape:  (100, 100, 50)
length of training data:  57
F:/Python/Dataset/NIMH/Fusion/dev/train\Month0\deface_1547_v1_pdw.nii.gz
Old shape:  (256, 256, 39)
New shape:  (100, 100, 50)
length of training data:  58
F:/Python/Dataset/NIMH/Fusion/dev/train\Month0\deface_1547_v1_t1w.nii.gz
Old shape:  (256, 256, 39)
New shape:  (100, 100, 50)
length of training data:  59
F:/Pytho

Old shape:  (192, 256, 44)
New shape:  (100, 100, 50)
length of training data:  106
F:/Python/Dataset/NIMH/Fusion/dev/train\Month3\deface_1507_v2_t2w.nii.gz
Old shape:  (192, 256, 44)
New shape:  (100, 100, 50)
length of training data:  107
F:/Python/Dataset/NIMH/Fusion/dev/train\Month3\deface_1508_v1_pdw.nii.gz
Old shape:  (192, 256, 44)
New shape:  (100, 100, 50)
length of training data:  108
F:/Python/Dataset/NIMH/Fusion/dev/train\Month3\deface_1508_v1_t1w.nii.gz
Old shape:  (192, 256, 44)
New shape:  (100, 100, 50)
length of training data:  109
F:/Python/Dataset/NIMH/Fusion/dev/train\Month3\deface_1508_v1_t2w.nii.gz
Old shape:  (192, 256, 44)
New shape:  (100, 100, 50)
length of training data:  110
F:/Python/Dataset/NIMH/Fusion/dev/train\Month3\deface_1510_v2_pdw.nii.gz
Old shape:  (192, 256, 44)
New shape:  (100, 100, 50)
length of training data:  111
F:/Python/Dataset/NIMH/Fusion/dev/train\Month3\deface_1510_v2_t1w.nii.gz
Old shape:  (192, 256, 44)
New shape:  (100, 100, 50)
leng

Old shape:  (192, 256, 46)
New shape:  (100, 100, 50)
length of training data:  159
F:/Python/Dataset/NIMH/Fusion/dev/train\Month9\deface_1470_v2_t1w.nii.gz
Old shape:  (192, 256, 46)
New shape:  (100, 100, 50)
length of training data:  160
F:/Python/Dataset/NIMH/Fusion/dev/train\Month9\deface_1470_v2_t2w.nii.gz
Old shape:  (192, 256, 46)
New shape:  (100, 100, 50)
length of training data:  161
F:/Python/Dataset/NIMH/Fusion/dev/train\Month9\deface_1480_v4_pdw.nii.gz
Old shape:  (192, 256, 48)
New shape:  (100, 100, 50)
length of training data:  162
F:/Python/Dataset/NIMH/Fusion/dev/train\Month9\deface_1480_v4_t1w.nii.gz
Old shape:  (192, 256, 48)
New shape:  (100, 100, 50)
length of training data:  163
F:/Python/Dataset/NIMH/Fusion/dev/train\Month9\deface_1480_v4_t2w.nii.gz
Old shape:  (192, 256, 48)
New shape:  (100, 100, 50)
length of training data:  164
F:/Python/Dataset/NIMH/Fusion/dev/train\Month9\deface_1481_v3_pdw.nii.gz
Old shape:  (192, 256, 44)
New shape:  (100, 100, 50)
leng

Old shape:  (192, 256, 48)
New shape:  (100, 100, 50)
length of training data:  212
F:/Python/Dataset/NIMH/Fusion/dev/train\Month9\deface_1543_v1_pdw.nii.gz
Old shape:  (192, 256, 48)
New shape:  (100, 100, 50)
length of training data:  213
F:/Python/Dataset/NIMH/Fusion/dev/train\Month9\deface_1543_v1_t1w.nii.gz
Old shape:  (192, 256, 48)
New shape:  (100, 100, 50)
length of training data:  214
F:/Python/Dataset/NIMH/Fusion/dev/train\Month9\deface_1543_v1_t2w.nii.gz
Old shape:  (192, 256, 48)
New shape:  (100, 100, 50)
length of training data:  215
F:/Python/Dataset/NIMH/Fusion/dev/train\Month9\deface_1545_v2_pdw.nii.gz
Old shape:  (256, 256, 51)
New shape:  (100, 100, 50)
length of training data:  216
F:/Python/Dataset/NIMH/Fusion/dev/train\Month9\deface_1545_v2_t1w.nii.gz
Old shape:  (256, 256, 51)
New shape:  (100, 100, 50)
length of training data:  217
F:/Python/Dataset/NIMH/Fusion/dev/train\Month9\deface_1545_v2_t2w.nii.gz
Old shape:  (256, 256, 51)
New shape:  (100, 100, 50)
leng

...

Old shape:  (256, 256, 47)
New shape:  (100, 100, 50)
length of training data:  316
F:/Python/Dataset/NIMH/Fusion/dev/train\Month12\deface_1531_v2_pdw.nii.gz
Old shape:  (192, 256, 46)
New shape:  (100, 100, 50)
length of training data:  317
F:/Python/Dataset/NIMH/Fusion/dev/train\Month12\deface_1531_v2_t1w.nii.gz
Old shape:  (192, 256, 46)
New shape:  (100, 100, 50)
length of training data:  318
F:/Python/Dataset/NIMH/Fusion/dev/train\Month12\deface_1531_v2_t2w.nii.gz
Old shape:  (192, 256, 46)
New shape:  (100, 100, 50)
length of training data:  319
F:/Python/Dataset/NIMH/Fusion/dev/train\Month12\deface_1533_v3_pdw.nii.gz
Old shape:  (192, 256, 48)
New shape:  (100, 100, 50)
length of training data:  320
F:/Python/Dataset/NIMH/Fusion/dev/train\Month12\deface_1533_v3_t1w.nii.gz
Old shape:  (192, 256, 48)
New shape:  (100, 100, 50)
length of training data:  321
F:/Python/Dataset/NIMH/Fusion/dev/train\Month12\deface_1533_v3_t2w.nii.gz
Old shape:  (192, 256, 48)
New shape:  (100, 100, 50

New shape:  (100, 100, 50)
length of training data:  368
F:/Python/Dataset/NIMH/Fusion/dev/train\Month24\deface_1484_v6_t1w.nii.gz
Old shape:  (192, 256, 48)
New shape:  (100, 100, 50)
length of training data:  369
F:/Python/Dataset/NIMH/Fusion/dev/train\Month24\deface_1484_v6_t2w.nii.gz
Old shape:  (192, 256, 48)
New shape:  (100, 100, 50)
length of training data:  370
F:/Python/Dataset/NIMH/Fusion/dev/train\Month24\deface_1494_v3_pdw.nii.gz
Old shape:  (256, 256, 48)
New shape:  (100, 100, 50)
length of training data:  371
F:/Python/Dataset/NIMH/Fusion/dev/train\Month24\deface_1494_v3_t1w.nii.gz
Old shape:  (256, 256, 47)
New shape:  (100, 100, 50)
length of training data:  372
F:/Python/Dataset/NIMH/Fusion/dev/train\Month24\deface_1494_v3_t2w.nii.gz
Old shape:  (256, 256, 48)
New shape:  (100, 100, 50)
length of training data:  373
F:/Python/Dataset/NIMH/Fusion/dev/train\Month24\deface_1506_v2_pdw.nii.gz
Old shape:  (256, 256, 50)
New shape:  (100, 100, 50)
length of training data: 

New shape:  (100, 100, 50)
length of training data:  420
F:/Python/Dataset/NIMH/Fusion/dev/train\Month36\deface_1442_v2_pdw.nii.gz
Old shape:  (192, 256, 51)
New shape:  (100, 100, 50)
length of training data:  421
F:/Python/Dataset/NIMH/Fusion/dev/train\Month36\deface_1442_v2_t1w.nii.gz
Old shape:  (192, 256, 51)
New shape:  (100, 100, 50)
length of training data:  422
F:/Python/Dataset/NIMH/Fusion/dev/train\Month36\deface_1442_v2_t2w.nii.gz
Old shape:  (192, 256, 51)
New shape:  (100, 100, 50)
length of training data:  423
F:/Python/Dataset/NIMH/Fusion/dev/train\Month36\deface_1442_v3_pdw.nii.gz
Old shape:  (192, 256, 51)
New shape:  (100, 100, 50)
length of training data:  424
F:/Python/Dataset/NIMH/Fusion/dev/train\Month36\deface_1442_v3_t1w.nii.gz
Old shape:  (192, 256, 51)
New shape:  (100, 100, 50)
length of training data:  425
F:/Python/Dataset/NIMH/Fusion/dev/train\Month36\deface_1442_v3_t2w.nii.gz
Old shape:  (192, 256, 51)
New shape:  (100, 100, 50)
length of training data: 

New shape:  (100, 100, 50)
length of training data:  472
F:/Python/Dataset/NIMH/Fusion/dev/train\Month36\deface_1473_v2_t2w.nii.gz
Old shape:  (192, 256, 66)
New shape:  (100, 100, 50)
length of training data:  473
F:/Python/Dataset/NIMH/Fusion/dev/train\Month36\deface_1478_v1_pdw.nii.gz
Old shape:  (192, 256, 51)
New shape:  (100, 100, 50)
length of training data:  474
F:/Python/Dataset/NIMH/Fusion/dev/train\Month36\deface_1478_v1_t1w.nii.gz
Old shape:  (192, 256, 51)
New shape:  (100, 100, 50)
length of training data:  475
F:/Python/Dataset/NIMH/Fusion/dev/train\Month36\deface_1478_v1_t2w.nii.gz
Old shape:  (192, 256, 51)
New shape:  (100, 100, 50)
length of training data:  476
F:/Python/Dataset/NIMH/Fusion/dev/train\Month36\deface_1483_v1_pdw.nii.gz
Old shape:  (192, 256, 51)
New shape:  (100, 100, 50)
length of training data:  477
F:/Python/Dataset/NIMH/Fusion/dev/train\Month36\deface_1483_v1_t1w.nii.gz
Old shape:  (192, 256, 51)
New shape:  (100, 100, 50)
length of training data: 

Old shape:  (192, 256, 72)
New shape:  (100, 100, 50)
length of training data:  525
F:/Python/Dataset/NIMH/Fusion/dev/train\Month36\deface_1515_v5_pdw.nii.gz
Old shape:  (192, 256, 60)
New shape:  (100, 100, 50)
length of training data:  526
F:/Python/Dataset/NIMH/Fusion/dev/train\Month36\deface_1515_v5_t1w.nii.gz
Old shape:  (192, 256, 60)
New shape:  (100, 100, 50)
length of training data:  527
F:/Python/Dataset/NIMH/Fusion/dev/train\Month36\deface_1515_v5_t2w.nii.gz
Old shape:  (192, 256, 60)
New shape:  (100, 100, 50)
length of training data:  528
F:/Python/Dataset/NIMH/Fusion/dev/train\Month36\deface_1515_v6_pdw.nii.gz
Old shape:  (192, 256, 60)
New shape:  (100, 100, 50)
length of training data:  529
F:/Python/Dataset/NIMH/Fusion/dev/train\Month36\deface_1515_v6_t1w.nii.gz
Old shape:  (192, 256, 60)
New shape:  (100, 100, 50)
length of training data:  530
F:/Python/Dataset/NIMH/Fusion/dev/train\Month36\deface_1515_v6_t2w.nii.gz
Old shape:  (192, 256, 60)
New shape:  (100, 100, 50

In [ ]:
# was borrowed from "https://theaisummer.com/medical-image-processing/"

import scipy
def resize_data_volume(data, dim):
    """
    Resize the data to the dim size
    """
    depth, height, width = data.shape
    scale = [dim[0] * 1.0 / depth, dim[1] * 1.0 / height, dim[2] * 1.0 / width]
    print(scale)
    return scipy.ndimage.interpolation.zoom(data, scale, order=0)

def resize_data_volume_by_scale(data, scale):
    """
    Resize the data based on the provided scale
    """
    scale_list = [scale,scale,scale]
    return scipy.ndimage.interpolation.zoom(data, scale_list, order=0)


result = resize_data_volume_by_scale(epi_img_numpy, 0.5)
result2 = resize_data_volume_by_scale(epi_img_numpy, 2)

show_mid_slice(epi_img_numpy)
show_mid_slice(result)
show_mid_slice(result2)
